In [1]:
import os
import glob
import numpy as np

from pathlib import Path
from aicsimageio import AICSImage, readers
from blimp.preprocessing.illumination_correction import IlluminationCorrection

data_dir = "/srv/scratch/berrylab/z3532965/NikonSpinningDisk/240305/240301_ARMC5KO_IF"
image_dir = os.path.join(data_dir,"20240305_184055_331/OME-TIFF-MIP")

illum_corr_file = os.path.join(data_dir,"/srv/scratch/berrylab/z3532965/NikonSpinningDisk/240305/ILLUMCORR/illumination_correction.pkl")

26-Jun-24 17:40:05 - bfio.backends - WARNING  - Java backend is not available. This could be due to a missing dependency (jpype).


# Apply illumination correction

Load illumination correction object

In [2]:
illumination_correction = IlluminationCorrection(from_file=illum_corr_file)

## Save examples for training nuclear segmentation model

In [3]:
training_dir = Path(image_dir).parent / 'TRAINING'
if not training_dir.exists(): 
    training_dir.mkdir()

In [4]:
example_image_files = glob.glob(str(Path(image_dir) / "*003.ome.tiff"))

In [5]:
for f in example_image_files:
    aics_image = AICSImage(Path(image_dir) / Path(f).name, reader=readers.ome_tiff_reader.OmeTiffReader)
    aics_image_corrected = illumination_correction.correct(aics_image)
    # crop 500 pixels from left edge and save DAPI (C=3)
    DAPI = AICSImage(aics_image_corrected.get_image_data('YX',Z=0,C=3,T=0)[np.newaxis,np.newaxis,np.newaxis,:,500:])
    DAPI.save(str(training_dir / Path(f).name))

In [6]:
examples_dir = Path(image_dir).parent / 'EXAMPLES'
if not examples_dir.exists(): 
    examples_dir.mkdir()

In [7]:
for f in example_image_files[1:4]:
    aics_image = AICSImage(Path(image_dir) / Path(f).name, reader=readers.ome_tiff_reader.OmeTiffReader)
    aics_image_corrected = illumination_correction.correct(aics_image)
    aics_image_corrected.save(examples_dir / Path(f).name)

In [8]:
examples_dir

PosixPath('/srv/scratch/berrylab/z3532965/NikonSpinningDisk/240305/240301_ARMC5KO_IF/20240305_184055_331/EXAMPLES')